In [3]:
import minari

dataset = minari.load_dataset('D4RL/relocate/human-v2')
print("Observation space:", dataset.observation_space)
print("Action space:", dataset.action_space)
print("Total episodes:", dataset.total_episodes)
print("Total steps:", dataset.total_steps)

Observation space: Box(-inf, inf, (39,), float64)
Action space: Box(-1.0, 1.0, (30,), float32)
Total episodes: 25
Total steps: 9942


In [29]:
expert_dataset = minari.load_dataset('D4RL/relocate/expert-v2')
print("Expert observation space:", expert_dataset.observation_space)
print("Expert action space:", expert_dataset.action_space)
print("Total expert episodes:", expert_dataset.total_episodes)
print("Total expert steps:", expert_dataset.total_steps)


Expert observation space: Box(-inf, inf, (39,), float64)
Expert action space: Box(-1.0, 1.0, (30,), float32)
Total expert episodes: 5000
Total expert steps: 1000000


In [ ]:
from scipy.interpolate import interp1d
import numpy as np

trajectories = np.zeros((25, 200, 30+39))

for i in range(25):
    episode = dataset[i]
    obs = episode.observations[:-1]
    actions = episode.actions

    T = np.linspace(0, 1, 200)
    obs_interp = interp1d(np.linspace(0, 1, len(obs)), obs, axis=0)(T)
    actions_interp = interp1d(np.linspace(0, 1, len(actions)), actions, axis=0)(T)

    trajectories[i, :, :30] = actions_interp
    trajectories[i, :, 30:] = obs_interp

In [ ]:
from scipy.interpolate import interp1d
import numpy as np

num_traj = 25
trajectories = np.zeros((num_traj, 200, 30+39))

for i in range(num_traj):
    episode = dataset[i]
    obs = episode.observations
    actions = episode.actions

    raw_episode_obs = np.zeros((len(obs), 39))
    raw_episode_act = np.zeros((len(actions)+1, 30))

    raw_episode_obs = obs
    raw_episode_act[1:] = actions

    print(f"Episode {i} - Observations shape: {raw_episode_obs.shape}, Actions shape: {raw_episode_act.shape}")

    raw_episode = np.concatenate((raw_episode_obs, raw_episode_act), axis=-1)

    T = np.linspace(0, 1, 200)
    episode_interp = interp1d(np.linspace(0, 1, len(raw_episode)), raw_episode, axis=0)(T)

    trajectories[i] = episode_interp

In [5]:
print("Expert trajectories shape:", trajectories.shape)
np.save('trajectories_v2.npy',trajectories)

Expert trajectories shape: (25, 200, 69)


New dataset

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
import pickle

absolute_initial_palm_pos = np.array([-0.00692036, -0.19996033, 0.15038709])
absolute_dataset = pickle.load(open('data/relocate.pickle', 'rb'))

num_traj = 25

new_context = np.zeros((num_traj, 12))
new_action_trajectories = np.zeros((num_traj, 200, 30))
new_observation_trajectories = np.zeros((num_traj, 200, 30))

for i in range(num_traj):
    episode = dataset[i]
    actions = episode.actions
    observations = episode.observations

    raw_episode_act = actions

    raw_episode_obs = np.zeros((len(observations), 30 + 12))
    raw_episode_obs[:, :30] = observations



    

    T = np.linspace(0, 1, 200)
    episode_act_interp = interp1d(np.linspace(0, 1, len(raw_episode_act)), raw_episode_act, axis=0)(T)
    episode_obs_interp = interp1d(np.linspace(0, 1, len(raw_episode_obs)), raw_episode_obs, axis=0)(T)
    
    new_action_trajectories[i] = episode_act_interp
    new_observation_trajectories[i] = episode_obs_interp

    initial_abs_dict = absolute_dataset[i]['init_state_dict']
    new_context[i, :3] = absolute_initial_palm_pos - initial_abs_dict['obj_pos']
    new_context[i, 3:6] = absolute_initial_palm_pos - initial_abs_dict['target_pos']
    new_context[i, 6:9] = initial_abs_dict['obj_pos']
    new_context[i, 9:12] = initial_abs_dict['target_pos']


np.save('new_observation_trajectories.npy', new_observation_trajectories)
np.save('new_action_trajectories.npy', new_action_trajectories)
np.save('new_diff_context.npy', new_context)